# Revenue Impacts

This chapter presents the quantitative results of the analysis of seven Social Security benefit taxation reform options. All estimates represent changes in federal tax revenue over the 10-year period 2026-2035. Positive values indicate revenue increases (deficit reducing), while negative values indicate revenue losses (deficit increasing).

In [ ]:
# Import necessary libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

# Define PolicyEngine's color palette
BLACK = "#000000"
BLUE_LIGHT = "#D8E6F3"
BLUE_PRIMARY = "#2C6496"
DARK_BLUE_HOVER = "#1d3e5e"
DARK_GRAY = "#616161"
DARKEST_BLUE = "#0C1A27"
GRAY = "#808080"
LIGHT_GRAY = "#F2F2F2"
MEDIUM_DARK_GRAY = "#D2D2D2"
MEDIUM_LIGHT_GRAY = "#BDBDBD"
WHITE = "#FFFFFF"

In [ ]:
# Load the policy impacts data from CSV
df = pd.read_csv("../data/policy_impacts.csv")

# Clean up reform names for display
reform_name_map = {
    "Full Repeal of Social Security Benefits Taxation": "Option 1: Full Repeal",
    "Taxation of 85% of Social Security Benefits": "Option 2: 85% Taxation",
    "85% Taxation with Permanent Senior Deduction Extension": "Option 3: 85% with Senior Deduction",
    "Social Security Tax Credit System ($500)": "Option 4: Tax Credit ($500)",
    "Roth-Style Swap": "Option 5: Roth-Style Swap",
    "Phased Roth-Style Swap": "Option 6: Phased Roth-Style",
    "Eliminate Bonus Senior Deduction": "Option 7: Eliminate Bonus Senior Deduction"
}

df['reform_display'] = df['reform_name'].map(reform_name_map)

# Convert revenue impact to billions for display
df['revenue_impact_billions'] = df['revenue_impact'] / 1e9

print(f"Loaded {len(df)} records from {len(df['reform_name'].unique())} reforms")
print(f"Years covered: {df['year'].min()}-{df['year'].max()}")

## Summary of 10-Year Budgetary Impacts

The microsimulation analysis shows the following budgetary impacts across the seven policy options:

In [ ]:
# Calculate 10-year totals for each reform
totals = df.groupby(['reform_name', 'reform_display'])['revenue_impact_billions'].sum().reset_index()
totals = totals.sort_values('revenue_impact_billions')

# Format the display
totals['formatted_impact'] = totals['revenue_impact_billions'].apply(lambda x: f"${x:,.1f}" if x >= 0 else f"-${abs(x):,.1f}")

# Display as table
summary_table = totals[['reform_display', 'formatted_impact']].rename(columns={
    'reform_display': 'Policy Option',
    'formatted_impact': '10-Year Impact (Billions)'
})

from IPython.display import display, HTML
html_table = summary_table.to_html(index=False, classes='dataframe', table_id='summary-table')
display(HTML(html_table))

## Year-by-Year Revenue Impact Visualization

In [ ]:
# Create interactive line chart of revenue impacts over time
fig = px.line(
    df,
    x='year',
    y='revenue_impact_billions',
    color='reform_display',
    title='Revenue Impact by Year (2026-2035)<br>All Reform Options',
    labels={
        'year': 'Year',
        'revenue_impact_billions': 'Revenue Impact (Billions $)',
        'reform_display': 'Reform Option'
    },
    color_discrete_sequence=px.colors.qualitative.Set2
)

# Add zero line
fig.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5)

# Update layout
fig.update_layout(
    font=dict(family="Roboto Serif"),
    yaxis_tickformat="$,.0f",
    font_color=BLACK,
    margin={"l": 50, "r": 50, "b": 100, "t": 100, "pad": 4},
    hovermode='x unified',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ),
    annotations=[
        {
            "x": 1,
            "y": -0.25,
            "xref": "paper",
            "yref": "paper",
            "text": "Source: PolicyEngine US Microsimulation",
            "showarrow": False,
            "font": {"family": "Roboto Serif", "size": 10, "color": DARK_GRAY},
        }
    ],
)

fig.show()

## Year-by-Year Net Effect on Tax Revenues (2026-2035)

**Net Effect on Tax Revenues by Year (Billions of Dollars)**

In [ ]:
# Create pivot table for year-by-year display
pivot = df.pivot_table(
    values='revenue_impact_billions',
    index='reform_display',
    columns='year',
    aggfunc='sum'
).round(1)

# Add 10-year total column
pivot['10-Year Total'] = pivot.sum(axis=1).round(1)

# Format for display
def format_value(x):
    if pd.isna(x) or x == 0:
        return "0.0"
    elif x < 0:
        return f"-{abs(x):.1f}"
    else:
        return f"{x:.1f}"

# Apply formatting
formatted_pivot = pivot.applymap(format_value)

# Make the 10-year total column bold
formatted_pivot['10-Year Total'] = formatted_pivot['10-Year Total'].apply(lambda x: f"**{x}**")

# Display the table
display(HTML(formatted_pivot.to_html(classes='dataframe', table_id='yearly-table')))

## Cumulative Revenue Impact Over Time

In [ ]:
# Calculate cumulative impacts
df_sorted = df.sort_values(['reform_name', 'year'])
df_sorted['cumulative_impact'] = df_sorted.groupby('reform_name')['revenue_impact_billions'].cumsum()

# Create cumulative impact chart
fig_cumulative = px.line(
    df_sorted,
    x='year',
    y='cumulative_impact',
    color='reform_display',
    title='Cumulative Revenue Impact (2026-2035)<br>All Reform Options',
    labels={
        'year': 'Year',
        'cumulative_impact': 'Cumulative Revenue Impact (Billions $)',
        'reform_display': 'Reform Option'
    },
    color_discrete_sequence=px.colors.qualitative.Set2
)

# Add zero line
fig_cumulative.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5)

# Update layout
fig_cumulative.update_layout(
    font=dict(family="Roboto Serif"),
    yaxis_tickformat="$,.0f",
    font_color=BLACK,
    margin={"l": 50, "r": 50, "b": 100, "t": 100, "pad": 4},
    hovermode='x unified',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ),
    annotations=[
        {
            "x": 1,
            "y": -0.25,
            "xref": "paper",
            "yref": "paper",
            "text": "Source: PolicyEngine US Microsimulation",
            "showarrow": False,
            "font": {"family": "Roboto Serif", "size": 10, "color": DARK_GRAY},
        }
    ],
)

fig_cumulative.show()

## Revenue Impact Comparison Bar Chart

In [ ]:
# Create bar chart comparing 10-year totals
fig_bar = px.bar(
    totals,
    x='revenue_impact_billions',
    y='reform_display',
    orientation='h',
    title='10-Year Revenue Impact Comparison<br>All Reform Options (2026-2035)',
    labels={
        'revenue_impact_billions': 'Revenue Impact (Billions $)',
        'reform_display': 'Reform Option'
    },
    color='revenue_impact_billions',
    color_continuous_scale=['#d62728', '#ff7f0e', '#ffff00', '#90EE90', '#2ca02c'],
    color_continuous_midpoint=0
)

# Add vertical line at zero
fig_bar.add_vline(x=0, line_dash="dash", line_color="black", opacity=0.5)

# Update layout
fig_bar.update_layout(
    font=dict(family="Roboto Serif"),
    xaxis_tickformat="$,.0f",
    font_color=BLACK,
    margin={"l": 150, "r": 50, "b": 100, "t": 100, "pad": 4},
    showlegend=False,
    annotations=[
        {
            "x": 1,
            "y": -0.25,
            "xref": "paper",
            "yref": "paper",
            "text": "Source: PolicyEngine US Microsimulation",
            "showarrow": False,
            "font": {"family": "Roboto Serif", "size": 10, "color": DARK_GRAY},
        }
    ],
)

# Hide color scale
fig_bar.update_coloraxes(showscale=False)

fig_bar.show()